In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import scipy.stats as stats
from sklearn.ensemble import RandomForestClassifier
import color as c

In [8]:
data = pd.read_csv('mil_expend.csv')
data.fillna(0, inplace=True)
data.isna().sum()
data.head()

,Name,Code,Type,Indicator Name,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,ABW,Country,Military expenditure (current USD),0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,Afghanistan,AFG,Country,Military expenditure (current USD),0.0,0.0,0.0,0.0,0.0,0.0,...,2.518695e+08,2.981469e+08,3.258070e+08,2.385834e+08,2.171941e+08,2.682271e+08,1.995186e+08,1.858783e+08,1.914071e+08,1.980863e+08
2,Angola,AGO,Country,Military expenditure (current USD),0.0,0.0,0.0,0.0,0.0,0.0,...,3.311193e+09,3.500795e+09,3.639496e+09,4.144635e+09,6.090752e+09,6.841864e+09,3.608299e+09,2.764055e+09,3.062873e+09,1.983614e+09
3,Albania,ALB,Country,Military expenditure (current USD),0.0,0.0,0.0,0.0,0.0,0.0,...,1.827369e+08,1.858932e+08,1.970068e+08,1.832047e+08,1.800155e+08,1.781204e+08,1.323507e+08,1.308532e+08,1.443827e+08,1.804887e+08
4,Andorra,AND,Country,Military expenditure (current USD),0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [11]:
bric_code = ['BRA', 'CHN', 'EGY', 'RUS', 'ZAF', 'ETH', 'IRN', 'ARE', 'IDN', 'IND']
brics = data[data['Code'].isin(bric_code)]
brics

,Name,Code,Type,Indicator Name,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
6,United Arab Emirates,ARE,Country,Military expenditure (current USD),0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.383635e+10,1.750470e+10,1.918176e+10,1.902410e+10,2.356106e+10,2.275507e+10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
27,Brazil,BRA,Country,Military expenditure (current USD),3.827298e+08,3.423397e+08,3.874490e+08,4.419996e+08,3.542279e+08,6.328690e+08,...,2.564881e+10,3.400294e+10,3.693621e+10,3.398701e+10,3.287479e+10,3.265961e+10,2.461770e+10,2.422475e+10,2.928305e+10,2.776643e+10
38,China,CHN,Country,Military expenditure (current USD),0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.060000e+11,1.160000e+11,1.380000e+11,1.570000e+11,1.800000e+11,2.010000e+11,2.140000e+11,2.160000e+11,2.280000e+11,2.500000e+11
65,Egypt,EGY,Country,Military expenditure (current USD),0.000000e+00,0.000000e+00,2.496978e+08,3.357997e+08,4.484996e+08,5.151995e+08,...,4.017404e+09,4.407286e+09,4.463974e+09,4.557749e+09,4.359834e+09,5.085121e+09,5.475491e+09,4.513023e+09,2.765588e+09,3.109998e+09
70,Ethiopia,ETH,Country,Military expenditure (current USD),1.533526e+07,1.855527e+07,2.018539e+07,2.728952e+07,3.614000e+07,4.290000e+07,...,3.396278e+08,3.036170e+08,3.324418e+08,3.665390e+08,3.450642e+08,4.144760e+08,4.425515e+08,4.901772e+08,5.031582e+08,4.966211e+08
104,Indonesia,IDN,Country,Military expenditure (current USD),0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.304459e+09,4.663366e+09,5.838026e+09,6.531098e+09,8.384029e+09,6.929255e+09,7.639095e+09,7.385409e+09,8.178144e+09,7.437197e+09
107,India,IND,Country,Military expenditure (current USD),6.817657e+08,7.483882e+08,1.065436e+09,1.795449e+09,1.986654e+09,2.125990e+09,...,3.872215e+10,4.609045e+10,4.963382e+10,4.721692e+10,4.740353e+10,5.091411e+10,5.129548e+10,5.663762e+10,6.455944e+10,6.651029e+10
110,Iran,IRN,Country,Military expenditure (current USD),9.867987e+07,1.059406e+08,1.099010e+08,1.287129e+08,1.554455e+08,2.000000e+08,...,1.258462e+10,1.356127e+10,1.427767e+10,1.649396e+10,1.199719e+10,9.901105e+09,1.058877e+10,1.226396e+10,1.393120e+10,1.319415e+10
200,Russian Federation,RUS,Country,Military expenditure (current USD),0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,5.153212e+10,5.872023e+10,7.023752e+10,8.146940e+10,8.835290e+10,8.469650e+10,6.641871e+10,6.924531e+10,6.652730e+10,6.138755e+10
261,South Africa,ZAF,Country,Military expenditure (current USD),6.999997e+07,1.137500e+08,1.861999e+08,1.889999e+08,2.715999e+08,2.894499e+08,...,3.592688e+09,4.188168e+09,4.594154e+09,4.489590e+09,4.118208e+09,3.892469e+09,3.488868e+09,3.169756e+09,3.638937e+09,3.639879e+09


In [18]:

nato_codes = [
    'ALB', 'BEL', 'BGR', 'CAN', 'HRV', 'CZE', 'DNK', 'EST', 'FIN', 'FRA', 'DEU', 'GRC', 'HUN', 'ISL', 'ITA', 'LVA',
    'LTU', 'LUX', 'MNE', 'NLD', 'MKD', 'NOR', 'POL', 'PRT', 'ROU', 'SVK', 'SVN', 'ESP', 'SWE', 'TUR', 'GBR', 'USA'
]
nato_data = data[data['Code'].isin(nato_codes)]
nato_data



,Name,Code,Type,Indicator Name,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
3,Albania,ALB,Country,Military expenditure (current USD),0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.827369e+08,1.858932e+08,1.970068e+08,1.832047e+08,1.800155e+08,1.781204e+08,1.323507e+08,1.308532e+08,1.443827e+08,1.804887e+08
15,Belgium,BEL,Country,Military expenditure (current USD),3.832202e+08,3.912188e+08,4.222208e+08,4.446013e+08,4.970592e+08,5.007221e+08,...,5.620670e+09,5.244721e+09,5.499371e+09,5.168998e+09,5.263165e+09,5.191509e+09,4.202063e+09,4.314102e+09,4.484653e+09,4.959692e+09
19,Bulgaria,BGR,Country,Military expenditure (current USD),0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,9.632542e+08,8.934675e+08,8.290318e+08,8.075293e+08,8.995840e+08,8.357170e+08,6.608479e+08,7.554065e+08,8.241870e+08,1.095591e+09
33,Canada,CAN,Country,Military expenditure (current USD),1.702443e+09,1.677821e+09,1.671314e+09,1.610092e+09,1.657457e+09,1.574705e+09,...,1.893623e+10,1.931569e+10,2.139372e+10,2.045211e+10,1.851573e+10,1.785372e+10,1.793764e+10,1.778278e+10,2.134337e+10,2.162060e+10
52,Czech Republic,CZE,Country,Military expenditure (current USD),0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.718560e+09,2.497903e+09,2.474313e+09,2.220610e+09,2.148784e+09,2.022883e+09,1.779887e+09,1.954935e+09,2.077722e+09,2.710018e+09
53,Germany,DEU,Country,Military expenditure (current USD),2.884518e+09,3.266496e+09,4.308248e+09,4.981004e+09,4.888249e+09,4.978559e+09,...,4.747007e+10,4.625552e+10,4.814035e+10,4.647087e+10,4.593054e+10,4.610267e+10,3.981258e+10,4.157949e+10,4.538172e+10,4.947063e+10
56,Denmark,DNK,Country,Military expenditure (current USD),1.611376e+08,1.708377e+08,2.245502e+08,2.390280e+08,2.553879e+08,2.857912e+08,...,4.337356e+09,4.503492e+09,4.518590e+09,4.422458e+09,4.216648e+09,4.056862e+09,3.364048e+09,3.592728e+09,3.764033e+09,4.228194e+09
68,Spain,ESP,Country,Military expenditure (current USD),2.328398e+08,2.393488e+08,3.087917e+08,3.165203e+08,3.868200e+08,4.394257e+08,...,2.017827e+10,1.971079e+10,1.969544e+10,1.886062e+10,1.724296e+10,1.717855e+10,1.518885e+10,1.401444e+10,1.604353e+10,1.824829e+10
69,Estonia,EST,Country,Military expenditure (current USD),0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.343409e+08,3.322312e+08,3.892376e+08,4.368529e+08,4.793145e+08,5.121190e+08,4.635688e+08,4.976534e+08,5.374402e+08,6.184861e+08
73,Finland,FIN,Country,Military expenditure (current USD),8.955762e+07,1.053510e+08,1.544033e+08,1.285765e+08,1.399106e+08,1.495724e+08,...,3.940585e+09,3.717181e+09,4.099506e+09,3.943240e+09,4.161140e+09,3.985508e+09,3.399135e+09,3.415008e+09,3.429968e+09,3.849014e+09


In [22]:
nato_join_years = {
    'ALB': 2009,
    'BEL': 1949,
    'BGR': 2004,
    'CAN': 1949,
    'HRV': 2009,
    'CZE': 1999,
    'DNK': 1949,
    'EST': 2004,
    'FIN': 2023,
    'FRA': 1949,
    'DEU': 1955,
    'GRC': 1952,
    'HUN': 1999,
    'ISL': 1949,
    'ITA': 1949,
    'LVA': 2004,
    'LTU': 2004,
    'LUX': 1949,
    'MNE': 2017,
    'NLD': 1949,
    'MKD': 2020,
    'NOR': 1949,
    'POL': 1999,
    'PRT': 1949,
    'ROU': 2004,
    'SVK': 2004,
    'SVN': 2004,
    'ESP': 1982,
    'SWE': 2024,
    'TUR': 1952,
    'GBR': 1949,
    'USA': 1949
}

In [23]:
brics_join_years = {
    'BRA': 2006, 
    'RUS': 2006, 
    'IND': 2006, 
    'CHN': 2006,  
    'ZAF': 2010,  
    'EGY': 2024,  
    'ETH': 2024,  
    'IDN': 2025,  
    'IRN': 2024, 
    'ARE': 2024   
}